In [ ]:
# import packages
import torch
import torch.nn as nn
from torchvision import datasets, transforms

In [ ]:
# 1. Digit Classification
transform = transforms.Compose([transforms.ToTensor()])
mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
image, label = mnist_test[0]

flat_image = image.view(1, 784)

mlp_model = nn.Sequential(
    nn.Linear(784, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

with torch.no_grad():
    logits = mlp_model(flat_image)
    probabilities = torch.softmax(logits, dim=1)
    prediction = torch.argmax(probabilities, dim=1)

print("--- MLP MNIST CLASSIFICATION TEST ---")
print(f"Actual Label: {label}")
print(f"Output Shape: {logits.shape}") # [1, 10]
print(f"Predicted Class: {prediction.item()}")
print(f"Confidence in Prediction: {probabilities.max().item():.4f}")

--- MLP MNIST CLASSIFICATION TEST ---
Actual Label: 7
Output Shape: torch.Size([1, 10])
Predicted Class: 2
Confidence in Prediction: 0.1085


In [ ]:
#Extracting 16 spatial feature maps from a color image.
transform = transforms.Compose([transforms.ToTensor()])
test_set = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
image, label = test_set[0]
image_batch = image.unsqueeze(0)

conv_layer = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3)

with torch.no_grad():
    output = conv_layer(image_batch)

print("--- CNN CIFAR-10 TEST ---")
print(f"Original Image Shape: {image.shape}")
print(f"After Conv Layer (16 filters): {output.shape}")

--- CNN CIFAR-10 TEST ---
Original Image Shape: torch.Size([3, 32, 32])
After Conv Layer (16 filters): torch.Size([1, 16, 30, 30])


In [ ]:
# Predicting t+1 based on a window of 10 previous steps.
t = torch.linspace(0, 50, 500)
series = torch.sin(t)

window_size = 10
test_window = series[:window_size].view(1, window_size, 1) # [Batch, Seq, Feature]

lstm = nn.LSTM(input_size=1, hidden_size=32, batch_first=True)
fc = nn.Linear(32, 1)

with torch.no_grad():
    out, _ = lstm(test_window)
    prediction = fc(out[:, -1, :])

print("\n--- RNN SINE WAVE TEST ---")
print(f"Input Sequence (first 3): {test_window.flatten()[:3].tolist()}")
print(f"Actual Next Value: {series[window_size].item():.4f}")
print(f"Model Prediction: {prediction.item():.4f}")


--- RNN SINE WAVE TEST ---
Input Sequence (first 3): [0.0, 0.10003281384706497, 0.19906212389469147]
Actual Next Value: 0.8426
Model Prediction: -0.1470


In [ ]:
# Creating context-aware embeddings for a 4-word sentence.
sentence = "Deep learning is powerful"
vocab = {"Deep": 0, "learning": 1, "is": 2, "powerful": 3}
tokens = torch.tensor([vocab[w] for w in sentence.split()]).unsqueeze(1) # [Seq_Len, Batch]

embed = nn.Embedding(num_embeddings=10, embedding_dim=16)
tf_layer = nn.TransformerEncoderLayer(d_model=16, nhead=4)

with torch.no_grad():
    embedded = embed(tokens)
    output = tf_layer(embedded)

print("\n--- TRANSFORMER TEXT TEST ---")
print(f"Input Sentence: '{sentence}'")
print(f"Token IDs: {tokens.flatten().tolist()}")
print(f"Output Context Vectors Shape: {output.shape}") # [4, 1, 16]


--- TRANSFORMER TEXT TEST ---
Input Sentence: 'Deep learning is powerful'
Token IDs: [0, 1, 2, 3]
Output Context Vectors Shape: torch.Size([4, 1, 16])


In [ ]:
# Compressing a digit to 20 numbers and rebuilding it.
mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())
real_digit, _ = mnist_test[0] # A '7'
real_digit_flat = real_digit.view(-1, 784)

encoder = nn.Linear(784, 20)
decoder = nn.Sequential(nn.Linear(20, 400), nn.ReLU(), nn.Linear(400, 784), nn.Sigmoid())

with torch.no_grad():
    latent_vector = encoder(real_digit_flat)
    reconstruction = decoder(latent_vector).view(1, 28, 28)

print("\n--- VAE MNIST RECONSTRUCTION TEST ---")
print(f"Input image pixels: {real_digit.sum().item():.2f}")
print(f"Reconstructed pixels: {reconstruction.sum().item():.2f}")


--- VAE MNIST RECONSTRUCTION TEST ---
Input image pixels: 72.37
Reconstructed pixels: 392.77
